In [1]:
# docker run -p 6333:6333 qdrant/qdrant

In [2]:
from embedder import sentence_embedder
from vector_db import qdrant_client, collection_name
from text_splitter import CustomTextSplitter
from retriever import CustomRetriever
from utils import folder_into_db_collection

/home/ilgiz/dev/RAG-On-Obsidian/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
folder_into_db_collection(
    '../data/KnowledgeStore',
    qdrant_client,
    CustomTextSplitter(chunk_size=1024, chunk_overlap=256),
    sentence_embedder)

Created a chunk of size 1106, which is longer than the specified 1024
Created a chunk of size 1182, which is longer than the specified 1024
Created a chunk of size 1619, which is longer than the specified 1024
Created a chunk of size 2187, which is longer than the specified 1024
Created a chunk of size 1629, which is longer than the specified 1024
Created a chunk of size 1464, which is longer than the specified 1024
Created a chunk of size 1213, which is longer than the specified 1024
Created a chunk of size 1176, which is longer than the specified 1024
Created a chunk of size 1330, which is longer than the specified 1024
Created a chunk of size 1227, which is longer than the specified 1024
Created a chunk of size 1571, which is longer than the specified 1024
Created a chunk of size 1190, which is longer than the specified 1024
Created a chunk of size 1175, which is longer than the specified 1024
Created a chunk of size 1951, which is longer than the specified 1024
Created a chunk of s

In [5]:
from pprint import pprint

retriever = CustomRetriever(
    qdrant_client=qdrant_client,
    embedder=sentence_embedder,
    collection_name=collection_name
    )

search_result = retriever.get_relevant_documents(
    "Чем отличается REST от SOAP?", limit=1)
for result in search_result:
    pprint(result)
    print('****')
    print(result.payload['filename'])
    print(result.payload['text'])

print('===' * 10)

search_result = retriever.get_relevant_documents(
    "В чем суть CAP-теоремы?", limit=4)
for result in search_result:
    print('****')
    print(result.payload['filename'])
    print(result.payload['text'])

ScoredPoint(id='25b2ab25-c160-4537-9609-26fd9feae85e', version=139, score=0.89453614, payload={'file_path': '../data/KnowledgeStore/REST.md', 'filename': 'REST.md', 'text': 'REST (REpresentational\xa0State\xa0Transfer) - \xa0архитектурный стиль \xa0взаимодействия компонентов распределённого приложения в\xa0сети.\nОн отличается от SOAP тем, что не является протоколом и стандартом.\n\n[#]+ Требования\n\n1. Модель клиент-сервер - разграничение на две стороны\n2. Stateless - вся необходимая информация находится в самом запросе/ответе\n3. Кэширование - реализуется на промежуточных серверах, необходимо для повышения производительности, сервер должен сам определить что кэшировать и как долго\n4. Унифицированный интерфейс - идентификация ресурсов по URI\n5. Уровневая структура - \n6. Код по требованию - сервер может передать код для клиента\n\n[#]+ Плохие варианты\n\n- Введение новых HTTP-методов\n- `/missiles/[id]?action=launch` - действия в параметре запроса\n- `/missiles/[id]/launch` - глаг